# 自定义层

doc:
https://keras.io/zh/layers/writing-your-own-keras-layers/

good examples
https://keunwoochoi.wordpress.com/2016/11/18/for-beginners-writing-a-custom-keras-layer/

very good examples: 
https://zhuanlan.zhihu.com/p/36436904

https://blog.csdn.net/u013084616/article/details/79295857

https://www.youtube.com/results?search_query=keras+custom+layer


core layer

In [2]:
from keras.layers import Lambda

Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [ ]:
# example 1
# add a x -> x^2 layer
model.add(Lambda(lambda x: x ** 2))

# example 2
# add a layer that returns the concatenation
# of the positive part of the input and
# the opposite of the negative part

def antirectifier(x):
    x -= K.mean(x, axis=1, keepdims=True)
    x = K.l2_normalize(x, axis=1)
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=1)

def antirectifier_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 2  # only valid for 2D tensors
    shape[-1] *= 2
    return tuple(shape)

model.add(Lambda(antirectifier,
                 output_shape=antirectifier_output_shape))

custome layer with trainable weights

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # 为该层创建一个可训练的权重
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        
        # 一定要在最后调用它
        super(MyLayer, self).build(input_shape)  
        
    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
# a more difficult example

class ParametricMel(Layer):

    def __init__(self, n_mels, n_freqs, sr, scale=24., init='mel', **kwargs):
        self.supports_masking = True
        self.scale = scale # scaling
        self.n_mels = n_mels
        if init == 'mel':
            self.means_init = np.array(_mel_frequencies(n_mels, fmin=0.0, fmax=sr/2), dtype='float32')
            stds = self.means_init[1:] - self.means_init[:-1]
            self.stds_init = 0.3 * np.hstack((stds[0:1], stds[:])) # 0.3: kinda make sense by the resulting images..

        self.center_freqs_init = [float(i)*sr/2/(n_freqs-1) for i in range(n_freqs)] # dft frequencies

        super(ParametricMel, self).__init__(**kwargs)

    def build(self, input_shape):
        self.means = K.variable(self.means_init, 
                                name='{}_means'.format(self.name))
        self.stds =  K.variable(self.stds_init, 
                                name='{}_stds'.format(self.name))
        
        self.center_freqs_init = np.array(self.center_freqs_init)[np.newaxis, :] # (1, n_freq)
        self.center_freqs_init = np.tile(self.center_freqs_init, (self.n_mels, 1)) # (n_mels, n_freq)
        self.center_freqs = K.variable(self.center_freqs_init,
                                       name='{}_center_freqs'.format(self.name))
        self.trainable_weights = [self.means, self.stds] # [self.means, self.stds]
        self.n_freq = input_shape[1]
        self.n_time = input_shape[2]
        print '--build--'

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.n_mels, input_shape[2])

    def call(self, x, mask=None):
        means = K.expand_dims(self.means, dim=1)
        stds = K.expand_dims(self.stds, dim=1)
        freq_to_mel = (self.scale * K.exp(-1. * K.square(self.center_freqs - means) \
                           / (2. * K.square(stds)))) \
                          / (np.sqrt(2. * np.pi).astype('float32') * stds)  # (n_mel, n_freq)
        out = K.dot(freq_to_mel, x) # (n_mel, None, n_time)
        return K.permute_dimensions(out, (1, 0, 2))

In [ ]:
class Scale(Layer):
    '''
    该层功能：
        通过向量元素依次相乘（Element wise multiplication）调整上层输出的形状。
        out = in * gamma + beta,
        gamma代表权重weights，beta代表偏置bias
       '''
    def __init__(self, weights=None, axis=-1, beta_init = 'zero', gamma_init = 'one', momentum = 0.9, **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.Zeros()
        self.gamma_init = initializers.Ones()
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)
        
        self.gamma = K.variable(self.gamma_init(shape), name='{}_gamma'.format(self.name))
        self.beta = K.variable(self.beta_init(shape), name='{}_beta'.format(self.name))

        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out
    
    def compute_output_shape(self, input_shape):
        pass
